## Newton para Sistemas não Lineares

O método de Newton para sistemas não lineares busca encontrar a solução de um sistema f(x) = 0 utilizando aproximações sucessivas e a inversa do Jacobiano.


In [1]:
import numpy as np
import sympy as sp
from tabulate import tabulate

In [15]:
def newton_sistemas(funcs, vars, x0, tol):
    """
    Resolve sistemas de equações não lineares pelo método de Newton.
    
    Parâmetros:
    funcs -- Lista de funções do sistema
    vars -- Lista de variáveis
    x0 -- Aproximação inicial (array)
    tol -- Erro de tolerância
    
    Retorna:
    Solução aproximada e tabela com as iterações
    """
    # Converte as funções simbólicas para funções numéricas
    J = sp.Matrix([[sp.diff(f, var) for var in vars] for f in funcs])  # Jacobiano
    J_inv = J.inv()  # Inversa do Jacobiano
    
    f_lambdify = sp.lambdify(vars, funcs, 'numpy')
    J_inv_lambdify = sp.lambdify(vars, J_inv, 'numpy')
    
    x = np.array(x0, dtype=float)
    tabela = []
    
    while True:
        F_val = np.array(f_lambdify(*x), dtype=float).flatten()
        J_inv_val = np.array(J_inv_lambdify(*x), dtype=float)
        
        delta_x = -J_inv_val @ F_val
        x_new = x + delta_x
        
        erro_absoluto = round(np.linalg.norm(delta_x), 4)
        erro_relativo = round(erro_absoluto / (np.linalg.norm(x_new) if np.linalg.norm(x_new) != 0 else 1), 4)
        
        tabela.append([*x, erro_absoluto, erro_relativo])
        
        if erro_absoluto <= tol:
            return np.round(x_new, 4), tabela
        
        x = x_new

In [19]:
# Entrada do usuário
n = int(input("Digite o número de equações/variáveis: "))
vars = sp.symbols(f'x1:{n+1}')
funcs = []

# print("Digite as funções do sistema:")
for i in range(n):
    funcs.append(sp.sympify(input(f"f{i+1}(x1, ..., xn) = ")))

x0 = [float(input(f"Digite x{i+1} inicial: ")) for i in range(n)]
tol = float(input("Digite o erro de tolerância: "))

solucao, tabela = newton_sistemas(funcs, vars, x0, tol)

# Exibindo tabela de resultados
headers = [f"x{i+1}" for i in range(n)] + ["Erro Absoluto", "Erro Relativo"]
print(tabulate(tabela, headers=headers, floatfmt=".4f"))
print(f"Solução aproximada: {solucao}")

    x1      x2      x3    Erro Absoluto    Erro Relativo
------  ------  ------  ---------------  ---------------
1.0000  1.0000  1.0000           0.5933           0.2674
0.9597  1.3452  1.4809           0.0425           0.0195
0.9513  1.3219  1.4462           0.0001           0.0000
Solução aproximada: [0.9513 1.3219 1.4462]
